In [ ]:
import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.distributions as TD
from zmq import device
import torch.optim as optim
from datetime import datetime
import functools
from tqdm import tqdm
import math

# Move model on GPU if available
enable_cuda = True
device = torch.device('cuda' if torch.cuda.is_available() and enable_cuda else 'cpu')

'''
This code reproduces the real data experiments using the CCLE data.
Preprocessing steps follow the code of W. Tansey at https://github.com/tansey/hrt.
And the following code is obtained from https://github.com/alexisbellot/GCIT.
'''


def load_ccle(drug_target='PLX4720', feature_type='both', normalize=False):
    '''
    :param drug target: specific drug we w ant to analyse
    :param normalize: normalize data
    :return: genetic features (mutations) as a 2d array for each cancer cell and corresponding drug response measured with Amax
    '''
    if feature_type in ['expression', 'both']:
        # Load gene expression
        expression = pd.read_csv('./ccle_data/expression.txt', delimiter='\t', header=2, index_col=1).iloc[:, 1:]
        expression.columns = [c.split(' (ACH')[0] for c in expression.columns]
        features = expression
    if feature_type in ['mutation', 'both']:
        # Load gene mutation
        mutations = pd.read_csv('./ccle_data/mutation.txt', delimiter='\t', header=2, index_col=1).iloc[:,1:]
        mutations = mutations.iloc[[c.endswith('_MUT') for c in mutations.index]]
        features = mutations
    if feature_type == 'both':
        # get cells having both expression and mutation data
        both_cells = set(expression.columns) & set(mutations.columns)
        z = {}
        for c in both_cells:
            exp = expression[c].values
            if len(exp.shape) > 1:
                exp = exp[:, 0]
            z[c] = np.concatenate([exp, mutations[c].values])
        both_df = pd.DataFrame(z, index=[c for c in expression.index] + [c for c in mutations.index])
        features = both_df

    print('Genetic features dimension = {} on {} cancer cells'.format(features.shape[0], features.shape[1]))

    # Get per-drug X and y regression targets
    response = pd.read_csv('./ccle_data/response.csv', header=0, index_col=[0, 2])

    # names of cell lines, there are 504
    cells = response.index.levels[0]
    # names of drugs, there are 24
    drugs = response.index.levels[1]

    X_drugs = [[] for _ in drugs]
    y_drugs = [[] for _ in drugs]

    # subset data to include only cells, mutations and response associated with chosen drug
    for j, drug in enumerate(drugs):
            if drug_target is not None and drug != drug_target:
                continue # return to beginning of the loop
            for i, cell in enumerate(cells):
                if cell not in features.columns or (cell, drug) not in response.index:
                    continue
                # all j empty except index that corresponds to target drug
                # for this j we iteratively append all the mutations on every cell
                X_drugs[j].append(features[cell].values) # store genetic features (mutations and expression) that appear in cells
                y_drugs[j].append(response.loc[(cell, drug), 'Amax']) # store response of the drug
            print('{}: Cell number = {}'.format(drug, len(y_drugs[j])))

    # convert to np array
    X_drugs = [np.array(x_i) for x_i in X_drugs]
    y_drugs = [np.array(y_i) for y_i in y_drugs]

    if normalize:
        X_drugs = [(x_i if (len(x_i) == 0) else (x_i - x_i.min(axis=0, keepdims=True)) /
                                                (x_i.max(axis=0, keepdims=True) - x_i.min(axis=0, keepdims=True)))
                   for x_i in X_drugs]
        y_drugs = [(y_i if (len(y_i) == 0 or y_i.std() == 0) else (y_i - y_i.min(axis=0, keepdims=True)) /
                                                (y_i.max(axis=0, keepdims=True) - y_i.min(axis=0, keepdims=True)))
                   for y_i in y_drugs]

    '''
    if normalize:
        X_drugs = [(x_i if (len(x_i) == 0) else (x_i - x_i.mean(axis=0, keepdims=True)) /
        x_i.std(axis=0).clip(1e-6)) for x_i in X_drugs]
        y_drugs = [(y_i if (len(y_i) == 0 or y_i.std() == 0) else (y_i - y_i.mean()) / y_i.std()) for y_i in y_drugs]
    '''
    drug_idx = drugs.get_loc(drug_target)
    # 2d array for features and 1d array for response
    X_drug, y_drug = X_drugs[drug_idx], y_drugs[drug_idx]

    return X_drug, y_drug, features


# X_drug, y_drug, features = load_ccle(feature_type='mutation')


def ccle_feature_filter(X, y, threshold=0.1):
    '''
    :param X: features
    :param y: response
    :param threshold: correlation threshold
    :return: logical array with False for all features that do not have at least pearson correlation at threshold with y
    and correlations for all variables
    '''
    corrs = np.array([np.abs(np.corrcoef(x, y)[0, 1]) if x.std() > 0 else 0 for x in X.T])
    selected = corrs >= threshold # True/False
    print(selected.sum(), selected.shape, corrs)
    return selected, corrs

# ccle_selected, corrs = ccle_feature_filter(X_drug, y_drug, threshold=0.1)

# features.index[ccle_selected]
# stats.describe(corrs[ccle_selected])


def fit_elastic_net_ccle(X, y, nfolds=3):
    '''
    :param X: features
    :param y: response
    :param nfolds: number of folds for hyperparameter tuning
    :return: fitted elastic net model
    '''
    from sklearn.linear_model import ElasticNetCV
    # The parameter l1_ratio corresponds to alpha in the glmnet R package
    # while alpha corresponds to the lambda parameter in glmnet
    # enet = ElasticNetCV(l1_ratio=np.linspace(0.2, 1.0, 10),
    #                     alphas=np.exp(np.linspace(-6, 5, 250)),
    #                     cv=nfolds)
    enet = ElasticNetCV(l1_ratio=0.2, # It always chooses l1_ratio=0.2
                        alphas=np.exp(np.linspace(-6, 5, 250)),
                        cv=nfolds)
    print('Fitting via CV')
    enet.fit(X, y)
    alpha, l1_ratio = enet.alpha_, enet.l1_ratio_
    print('Chose values: alpha={}, l1_ratio={}'.format(alpha, l1_ratio))
    return enet

# elastic_model = fit_elastic_net_ccle(X_drug[:,ccle_selected], y_drug)


def fit_random_forest_ccle(X, y):
    '''
    :param X: features
    :param y: response
    :param nfolds: number of folds for hyperparameter tuning
    :return: fitted elastic net model
    '''
    from sklearn.ensemble import RandomForestRegressor

    rf = RandomForestRegressor()

    rf.fit(X,y)

    return rf

# rf_model = fit_random_forest_ccle(X_drug[:,ccle_selected], y_drug)


def plot_ccle_predictions(model, X, y):
    from sklearn.metrics import r2_score
    plt.close()
    y_hat = model.predict(X)
    plt.scatter(y_hat, y, color='blue')
    plt.plot([min(y.min(), y_hat.min()), max(y.max(), y_hat.max())],
             [min(y.min(), y_hat.min()),max(y.max(), y_hat.max())], color='red', lw=3)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title(' ($r^2$={:.4f})'.format( r2_score(y, y_hat)))
    plt.tight_layout()

# plot_ccle_predictions(elastic_model,X_drug[:,ccle_selected],y_drug)


def print_top_features(model):
    # model_weights = np.mean([m.coef_ for m in model.models], axis=0)
    if model == rf_model:
        model_weights = model.feature_importances_
    else:
        model_weights = model.coef_

    ccle_features = features[ccle_selected]

    print('Top by fit:')
    for idx, top in enumerate(np.argsort(np.abs(model_weights))[::-1]):
        print('{}. {}: {:.4f}'.format(idx+1, ccle_features.index[top], model_weights[top]))

# print_top_features(rf_model)
# print_top_features(elastic_model)


def get_p_value_stat_1(boot_num, M, n, gen_x_all_torch, gen_y_all_torch, x_torch, y_torch, z_torch, sigma_w, sigma_u=1,
            sigma_v=1, boor_rv_type="gaussian", dy_g=1, dx_g=1):
  """
    Compute the p-value

    Input:
    - boot_num: Integer giving the number of bootstrap samples.
    - M: Integer giving the number of training samples per batch.
    - n: Integer giving the number of training samples.
    - gen_x_all_torch: PyTorch Tensor (batch_size, M) of generated data of X.
    - gen_y_all_torch: PyTorch Tensor (batch_size, M) of generated data of Y.
    - x_torch: PyTorch Tensor (batch_size) of training input X.
    - y_torch: PyTorch Tensor (batch_size) of training input Y.
    - z_torch: PyTorch Tensor (batch_size, dimension_Z) of training input Z
    - sigma_w: Float of the bandwith of the Laplace kernel.
    - sigma_u: Float of the bandwith of the Laplace kernel.
    - sigma_v: Float of the bandwith of the Laplace kernel.
    - boor_rv_type: "rademacher" or "gaussian", specifying the reference distribution.

    Output:
    - p_value: Float giving the p-value.
  """

    w_mx = torch.linalg.vector_norm(z_torch.repeat(n, 1, 1) - torch.swapaxes(z_torch.repeat(n, 1, 1), 0, 1), ord=1, dim=2)
    w_mx = torch.exp(-w_mx / sigma_w)

    u_mx_1 = torch.exp(-torch.linalg.vector_norm(y_torch.repeat(n, 1, 1) - torch.swapaxes(y_torch.repeat(n, 1, 1), 0, 1), ord=1, dim=2) / sigma_u)
    u_mx_2 = torch.mean(
        torch.exp(-torch.linalg.vector_norm(gen_y_all_torch.repeat(n, 1, 1).reshape(n, n, -1, dy_g) - y_torch.repeat(1, n).reshape(n, n, 1, dy_g), ord=1, dim=3) / sigma_u), dim=2)
    u_mx_3 = u_mx_2.T

    gen_y_all_torch_rep = gen_y_all_torch.repeat(n, 1, 1).reshape(n, n, -1, dy_g)

    temp_mx = torch.swapaxes(gen_y_all_torch_rep[:, :, 0, :], 0, 1)
    sum_mx = torch.mean(torch.exp(-torch.linalg.vector_norm(gen_y_all_torch_rep - temp_mx.reshape(n, n, 1, dy_g), ord=1, dim=3) / sigma_u), dim=2)

    v_mx_1 = torch.exp(-torch.linalg.vector_norm(x_torch.repeat(n, 1, 1) - torch.swapaxes(x_torch.repeat(n, 1, 1), 0, 1), ord=1, dim=2) / sigma_v)
    v_mx_2 = torch.mean(
        torch.exp(-torch.linalg.vector_norm(gen_x_all_torch.repeat(n, 1, 1).reshape(n, n, -1, dx_g) - x_torch.repeat(1, n).reshape(n, n, 1, dx_g), ord=1, dim=3) / sigma_v), dim=2)
    v_mx_3 = v_mx_2.T

    gen_x_all_torch_rep = gen_x_all_torch.repeat(n, 1, 1).reshape(n, n, -1, dx_g)

    temp2_mx = torch.swapaxes(gen_x_all_torch_rep[:, :, 0, :], 0, 1)
    sum2_mx = torch.mean(torch.exp(-torch.linalg.vector_norm(gen_x_all_torch_rep - temp2_mx.reshape(n, n, 1, dx_g), ord=1, dim=3) / sigma_v), dim=2)

    for i in range(1, M):
        temp_mx = torch.swapaxes(gen_y_all_torch_rep[:, :, i, :], 0, 1)
        temp_add_mx = torch.mean(torch.exp(-torch.linalg.vector_norm(gen_y_all_torch_rep - temp_mx.reshape(n, n, 1, dy_g), ord=1, dim=3) / sigma_u), dim=2)
        sum_mx = sum_mx + temp_add_mx

        temp2_mx = torch.swapaxes(gen_x_all_torch_rep[:, :, i, :], 0, 1)
        temp2_add_mx = torch.mean(torch.exp(-torch.linalg.vector_norm(gen_x_all_torch_rep - temp2_mx.reshape(n, n, 1, dx_g), ord=1, dim=3) / sigma_v), dim=2)
        sum2_mx = sum2_mx + temp2_add_mx

    u_mx_4 = 1 / M * sum_mx
    u_mx = u_mx_1 - u_mx_2 - u_mx_3 + u_mx_4
    v_mx_4 = 1 / M * sum2_mx
    v_mx = v_mx_1 - v_mx_2 - v_mx_3 + v_mx_4

    FF_mx = u_mx * v_mx * w_mx * (1 - torch.eye(n).to(device))

    stat = 1 / (n - 1) * torch.sum(FF_mx).item()

    # print("U_mx:", u_mx)
    # print("V_mx:", v_mx)
    # print("W_mx:", w_mx)
    boottemp = np.array([])
    if boor_rv_type == "rademacher":
        eboot = torch.sign(torch.randn(n, boot_num)).to(device)
    elif boor_rv_type == "gaussian":
        eboot = torch.randn(n, boot_num).to(device)
    for bb in range(boot_num):
        random_mx = torch.matmul(eboot[:, bb].reshape(-1, 1), eboot[:, bb].reshape(-1, 1).T)
        bootmatrix = FF_mx * random_mx
        stat_boot = 1 / (n - 1) * torch.sum(bootmatrix).item()
        boottemp = np.append(boottemp, stat_boot)
    return stat, boottemp


def find_loss(y_torch, gen_y_all_torch, z_torch, sigma_w, sigma_u, M, dim_y):
    """
    Compute the MMD loss via Laplace kernel.

    Inputs:
    - y_torch: PyTorch Tensor (batch_size) of training input. (X or Y)
    - gen_y_all_torch: PyTorch Tensor (batch_size, M) of generated data.
    - z_torch: PyTorch Tensor (batch_size, dimension_Z) of training input Z.
    - sigma_w: Float of the bandwith of the kernel.
    - sigma_u: Float of the bandwith of the kernel.
    - M: Number of training samples per batch.

    Outputs:
    - loss: PyTorch Tensor containing the MMD loss.
    """
    n = z_torch.shape[0]
    w_mx = torch.linalg.vector_norm(z_torch.repeat(n, 1, 1) - torch.swapaxes(z_torch.repeat(n, 1, 1), 0, 1), ord=1, dim=2)
    w_mx = torch.exp(-w_mx / sigma_w)

    u_mx_1 = torch.exp(-torch.linalg.vector_norm(y_torch.repeat(n, 1, 1) - torch.swapaxes(y_torch.repeat(n, 1, 1), 0, 1), ord=1, dim=2) / sigma_u)
    u_mx_2 = torch.mean(
        torch.exp(-torch.linalg.vector_norm(gen_y_all_torch.repeat(n, 1, 1).reshape(n, n, -1, dim_y) - y_torch.repeat(1, n).reshape(n, n, 1, dim_y), ord=1, dim=3) / sigma_u), dim=2)
    u_mx_3 = u_mx_2.T

    gen_y_all_torch_rep = gen_y_all_torch.repeat(n, 1, 1).reshape(n, n, -1, dim_y)
    temp_mx = torch.swapaxes(gen_y_all_torch_rep[:, :, 0, :], 0, 1)
    sum_mx = torch.mean(torch.exp(-torch.linalg.vector_norm(gen_y_all_torch_rep - temp_mx.reshape(n, n, 1, dim_y), ord=1, dim=3) / sigma_u), dim=2)

    for i in range(1, M):
        temp_mx = torch.swapaxes(gen_y_all_torch_rep[:, :, i, :], 0, 1)
        temp_add_mx = torch.mean(torch.exp(-torch.linalg.vector_norm(gen_y_all_torch_rep - temp_mx.reshape(n, n, 1, dim_y), ord=1, dim=3) / sigma_u), dim=2)
        sum_mx = sum_mx + temp_add_mx

    u_mx_4 = 1 / M * sum_mx
    u_mx = u_mx_1 - u_mx_2 - u_mx_3 + u_mx_4

    FF_mx = u_mx * w_mx * (1 - torch.eye(n).to(device))

    loss = 1 / (n) * torch.sum(FF_mx)
    return loss


##### GAN architecture #####

class Generator(torch.nn.Module):
    """
    Specify the neural network architecture of the Generator.

    Here, we consider a FNN with a fully connected hidden layer with a width of 50,
    which is followed by a Leaky ReLU activation. The coefficient of Leaky ReLU needs to be
    specified. Batch normalization may be added prior to the activation function.
    The output layer a fully connected layer without activation.

    Inputs:
    - input_dimension: Integer giving the dimension of input Z.
    - output_dimension: Integer giving the dimension of output X or Y.
    - noise_dimension: Integer giving the dimension of random noise.
    - hidden_layer_size: Integer giving the size of the hidden layer of the generator.
    - BN_type: 'True' or 'False' specifying whether batch normalization is included.
    - ReLU_coef: Scalar giving the coefficient of the Leaky ReLU layer.
    - drop_out_p: Float giving the dropout probability.
    - drop_input: Boolean specifying whether to add dropout to the input layer.

    Returns:
    - x: PyTorch Tensor containing the (output_dimension,) output of the generator.
    """

    def __init__(self, input_dimension, output_dimension, noise_dimension, hidden_layer_size, BN_type, ReLU_coef, drop_out_p,
                 drop_input = False):
      super(Generator, self).__init__()
      self.BN_type = BN_type
      self.ReLU_coef = ReLU_coef
      self.fc1 = torch.nn.Linear(input_dimension + noise_dimension, hidden_layer_size, bias=True)
      if BN_type:
        self.BN1 = torch.nn.BatchNorm1d(hidden_layer_size, 0.8, affine=False)
        self.BN2 = torch.nn.BatchNorm1d(hidden_layer_size, 0.8, affine=False)
        self.BN3 = torch.nn.BatchNorm1d(hidden_layer_size, 0.8, affine=False)
      self.leakyReLU1 = torch.nn.LeakyReLU(ReLU_coef)
      self.fc2 = torch.nn.Linear(hidden_layer_size, hidden_layer_size, bias=True)
      self.fc3 = torch.nn.Linear(hidden_layer_size, hidden_layer_size, bias=True)
      self.fc_last = torch.nn.Linear(hidden_layer_size, output_dimension, bias=True)
      self.sigmoid = torch.nn.Sigmoid()
      self.drop_out0 = torch.nn.Dropout(p=drop_out_p)
      self.drop_out1 = torch.nn.Dropout(p=drop_out_p)
      self.drop_out2 = torch.nn.Dropout(p=drop_out_p)
      self.drop_out3 = torch.nn.Dropout(p=drop_out_p)
      self.drop_input = drop_input
      self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
      if self.BN_type:
        if self.drop_input:
            x = self.drop_out0(x)
        x = self.drop_out1(self.leakyReLU1(self.BN1(self.fc1(x))))
        x = self.drop_out2(self.leakyReLU1(self.BN2(self.fc2(x))))
        # x = self.drop_out3(self.leakyReLU1(self.BN3(self.fc3(x))))
        x = self.fc_last(x)
        x = self.sigmoid(x)
      else:
        if self.drop_input:
            x = self.drop_out0(x)
        x = self.drop_out1(self.leakyReLU1(self.fc1(x)))
        x = self.drop_out2(self.leakyReLU1(self.fc2(x)))
        # x = self.drop_out3(self.leakyReLU1(self.fc3(x)))
        x = self.fc_last(x)
        x = self.sigmoid(x)
      return x
##### Auxilliary functions #####

def sample_noise(sample_size, noise_dimension, noise_type, input_var):
    """
    Generate a PyTorch Tensor of random noise from the specified reference distribution.

    Input:
    - sample_size: the sample size of noise to generate.
    - noise_dimension: the dimension of noise to generate.
    - noise_type: "normal", "unif" or "Cauchy", giving the reference distribution.

    Output:
    - A PyTorch Tensor of shape (sample_size, noise_dimension).
    """

    if (noise_type == "normal"):
      noise_generator = TD.MultivariateNormal(
        torch.zeros(noise_dimension).to(device), input_var * torch.eye(noise_dimension).to(device))

      Z = noise_generator.sample((sample_size,))
    if (noise_type == "unif"):
      Z = torch.rand(sample_size, noise_dimension)
    if (noise_type == "Cauchy"):
      Z = TD.Cauchy(torch.tensor([0.0]), torch.tensor([1.0])).sample((sample_size, noise_dimension)).squeeze(2)

    return Z

class DatasetSelect_GAN_ver2(torch.utils.data.Dataset):
  """
    Create a DatasetSelect object to generate the DataLoader in the learning process.

    Input:
    - X: PyTorch Tensor of shape (N, input_dimension) giving the training data of X.
    - Y: PyTorch Tensor of shape (N, output_dimension) giving the training data of Y.
    - batch_size: Integer giving the batch size.
  """

  def __init__(self, Y, Z, batch_size):
    self.Y_real = Y
    self.Z_real = Z
    self.batch_size = batch_size
    self.sample_size = Z.shape[0]

  def __len__(self):
    return self.sample_size

  def __getitem__(self, index):
    return self.Y_real[index], self.Z_real[index]


def train_label(Z, Y, rounded, noise_dimension, noise_type, G_lr, hidden_layer_size,
      DataLoader, BN_type, ReLU_coef,
      epochs_num=10,  sigma_z = 1, sigma_y = 1,
      lambda_1 = 1, wgt_decay = 0,
      lambda_3 = 0, drop_out_p = 0.2, M_train = 3,
      verbose = False):
    """
    Train loop for GAN.

    Inputs:
    - X: PyTorch Tensor (sample_size, dimension_X) of training input.
    - Y: PyTorch Tensor (sample_size, dimension_Y) of training input.
    - Z: PyTorch Tensor (sample_size, dimension_Z) of training input.
    - X_test: PyTorch Tensor (sample_size, dimension_X) of test input.
    - Y_test: PyTorch Tensor (sample_size, dimension_Y) of test input.
    - Z_test: PyTorch Tensor (sample_size, dimension_Z) of test input.
    - noise_dimension: Integer giving the dimension of random noise Z.
    - noise_type: "normal", "unif" or "Cauchy", giving the reference distribution.
    - G_lr: Float giving the learning rate of the generator.
    - hidden_layer_size: Integer giving the size of the hidden layer of the generator.
    - DataLoader: DataLoader object used to generate training batches.
    - BN_type: 'True' or 'False' specifying whether batch normalization is included.
    - ReLU_coef: Float giving the coefficient of the Leaky ReLU layer.
    - epochs_num: Number of epochs over the training dataset to use for training.
    - sigma_z: Float of the bandwith of the kernel.
    - sigma_x: Float of the bandwith of the kernel.
    - sigma_y: Float of the bandwith of the kernel.
    - normal_ini: Boolean specifying whether to initialize the generator with normal initialization.
    - lambda_1: Float giving the coefficient of the MMD loss using Laplace kernel.
    - lambda_2: Float giving the coefficient of the MMD loss using Gaussian kernel. (not using)
    - using_Gen: '1' or '2' specifying whether to use the first or second generator.(not using)
    - wgt_decay: Float giving the weight decay. (L2 regularization)
    - lambda_3: Scalar giving the coefficient of the L1 regularization.
    - drop_out_p: Float giving the dropout probability.
    - M_train: Number of training samples per batch used in the Laplace or Gaussian kernel.

    Outputs:
    - G_zy: PyTorch Net giving the trained generator.
    """

    input_dimension = Z.shape[1]
    output_dimension_y = Y.shape[1]
    M_eval = 50

    train_yz = DatasetSelect_GAN_ver2(Y, Z, 1)
    DataLoader_eval = torch.utils.data.DataLoader(train_yz, batch_size=1, shuffle=True)

    G_zy = Generator(input_dimension, output_dimension_y, noise_dimension, hidden_layer_size, BN_type, ReLU_coef, drop_out_p).to(device)
    G_zy_solver = optim.Adam(G_zy.parameters(), lr=G_lr, betas=(0.5, 0.999), weight_decay=wgt_decay) # betas=(0.5, 0.999),
    # G_zy_solver = optim.SGD(G_zy.parameters(), lr=G_lr, weight_decay=wgt_decay)

    G_zy = G_zy.eval()


    test_size = Z.shape[0]
    gen_y_all = torch.zeros(test_size, M_eval)
    z_all = torch.zeros(test_size, input_dimension)
    y_all = torch.zeros(test_size, output_dimension_y)

    cur_itr = 0

    for i, (y_test, z_test) in enumerate(DataLoader_eval):

      z_test_temp = z_test.repeat(M_eval,1).to(device)
      Noise_fake = sample_noise(z_test_temp.size()[0], noise_dimension, noise_type, input_var = 1.0/3.0).to(device)
      fake_y = G_zy(torch.cat((z_test_temp, Noise_fake),dim=1)).reshape(1, -1)
      gen_y_all[cur_itr,:] = fake_y.detach().reshape(-1)
      y_all[cur_itr,:] = y_test
      z_all[cur_itr,:] = z_test
      cur_itr = cur_itr + 1
    gen_y_mean = torch.mean(gen_y_all, dim = 1).reshape(-1,1)
    if rounded:
      gen_y_mean = (gen_y_mean> 0.9).float()

    mse_y = torch.mean((gen_y_mean - y_all)**2).item()

    # print(f'Epoch [0], MSE y [{mse_y}]')

    Y_real = Y.to(device)
    Z_real = Z.to(device)

    batch_size = Z_real.shape[0]
    Z_real_repeat = Z_real.repeat(M_eval,1)
    # Generate fake data
    Noise_fake = sample_noise(Z_real_repeat.shape[0], noise_dimension, noise_type, input_var = 1.0/3.0).to(device)
    Y_fake = G_zy(torch.cat((Z_real_repeat,Noise_fake),dim=1)).to(device)

    Y_fake = Y_fake.reshape(batch_size, M_eval, output_dimension_y)
    if rounded:
      Y_fake = (Y_fake> 0.9).float()
    MMD_loss = find_loss(Y_real, Y_fake, Z_real, sigma_z, sigma_y, M_eval, output_dimension_y)

    if verbose:
        print(f'Epoch [0],MSE [{mse_y}] MMD_loss [{MMD_loss.item()}]')

    iter_count = 0
    G_zy = G_zy.train()
    for epoch in tqdm(range(epochs_num)):
        # print('EPOCH: ', (epoch+1))
        batch_count = 0
        G_zy = G_zy.train()
        for Y_real, Z_real in DataLoader:
            Y_real = Y_real.to(device)
            Z_real = Z_real.to(device)

            batch_size = Z_real.shape[0]
            Z_real_repeat = Z_real.repeat(M_train,1)
            # Generate fake data
            Noise_fake = sample_noise(Z_real_repeat.shape[0], noise_dimension, noise_type, input_var = 1.0/3.0).to(device)
            Y_fake = G_zy(torch.cat((Z_real_repeat,Noise_fake),dim=1)).to(device)

            Y_fake = Y_fake.reshape(batch_size, M_train, output_dimension_y)

            # standardise = True

            # if standardise:
            #     Y_fake = (Y_fake - torch.mean(Y_fake, dim=0, keepdim=True)) / torch.std(Y_fake, dim=0, keepdim=True)
            #     Y_real = (Y_real - torch.mean(Y_real, dim=0, keepdim=True)) / torch.std(Y_real, dim=0, keepdim=True)
            #     Z_real = (Z_real - torch.mean(Z_real, dim=0, keepdim=True)) / torch.std(Z_real, dim=0, keepdim=True)

            # Generator step
            g_zy_error = None
            G_zy_solver.zero_grad()

            l1_regularization = 0

            for param in G_zy.parameters():
                l1_regularization += torch.linalg.vector_norm(param, ord = 1)

            g_zy_error = lambda_1 * find_loss(Y_real, Y_fake, Z_real, sigma_z, sigma_y, M_train, output_dimension_y) + lambda_3 * l1_regularization

            g_zy_error.backward()
            torch.nn.utils.clip_grad_norm_(G_zy.parameters(), max_norm=0.5)
            G_zy_solver.step()

            iter_count += 1
            batch_count += 1

        G_zy = G_zy.eval()
        if((epoch+1) % 1000 == 0) and verbose:

          test_size = Z.shape[0]
          gen_y_all = torch.zeros(test_size, M_eval)
          z_all = torch.zeros(test_size, input_dimension)
          y_all = torch.zeros(test_size, output_dimension_y)

          cur_itr = 0

          for i, (y_test, z_test) in enumerate(DataLoader_eval):

            z_test_temp = z_test.repeat(M_eval,1).to(device)
            Noise_fake = sample_noise(z_test_temp.size()[0], noise_dimension, noise_type, input_var = 1.0/3.0).to(device)
            fake_y = G_zy(torch.cat((z_test_temp, Noise_fake),dim=1)).reshape(1, -1)
            gen_y_all[cur_itr,:] = fake_y.detach().reshape(-1)
            y_all[cur_itr,:] = y_test
            z_all[cur_itr,:] = z_test
            cur_itr = cur_itr + 1
          gen_y_mean = torch.mean(gen_y_all, dim = 1).reshape(-1,1)
          if rounded:
            gen_y_mean = (gen_y_mean> 0.9).float()
          mse_y = torch.mean((gen_y_mean - y_all)**2).item()

          # print(f'Epoch [{epoch+1}/{epochs_num}], MSE y [{mse_y}]')

          Y_real = Y.to(device)
          Z_real = Z.to(device)

          batch_size = Z_real.shape[0]
          Z_real_repeat = Z_real.repeat(M_eval,1)
          # Generate fake data
          Noise_fake = sample_noise(Z_real_repeat.shape[0], noise_dimension, noise_type, input_var = 1.0/3.0).to(device)
          Y_fake = G_zy(torch.cat((Z_real_repeat,Noise_fake),dim=1)).to(device)

          Y_fake = Y_fake.reshape(batch_size, M_eval, output_dimension_y)
          if rounded:
            Y_fake = (Y_fake> 0.9).float()
          MMD_loss = find_loss(Y_real, Y_fake, Z_real, sigma_z, sigma_y, M_eval, output_dimension_y)

          if verbose:
              print(f'Epoch [{epoch+1}/{epochs_num}], MSE [{mse_y}] MMD_loss [{MMD_loss.item()}]')



    return G_zy



In [ ]:
def mgan_ccle(ccle_index_input, ccle_threshold_input, batch_size,
        noise_dimension, G_zy_lr, G_zx_lr, k, M, boot_num,
        boor_rv_type, set_seed = 42, verbose = False, epcoh_y = 4000,
        epcoh_x = 4000):
    x_drug, y, features = load_ccle(feature_type='mutation', normalize=False)

    ccle_selected, corrs = ccle_feature_filter(x_drug, y, threshold=ccle_threshold_input)

    features.index[ccle_selected]

    var_names = ['BRAF.MC_MUT', 'BRAF.V600E_MUT', 'HIP1_MUT', 'CDC42BPA_MUT', 'THBS3_MUT', 'DNMT1_MUT', 'PRKD1_MUT',
                  'FLT3_MUT', 'PIP5K1A_MUT', 'MAP3K5_MUT']
    idx = []

    for var in var_names:
        idx.append(features.T.columns.get_loc(var))

    x = x_drug[:, idx[ccle_index_input]]
    ccle_selected[idx] = True
    ccle_selected[idx[ccle_index_input]] = False
    z = x_drug[:, ccle_selected]

    z_dim = z.shape[1]
    x = np.expand_dims(x, axis=1).astype(np.float64)
    y = np.expand_dims(y, axis=1).astype(np.float64)
    n = y.shape[0]

    z = (z - z.min()) / (z.max() - z.min())
    x = (x - x.min()) / (x.max() - x.min())
    y = (y - y.min()) / (y.max() - y.min())

    x, y, z = torch.from_numpy(x).float(), torch.from_numpy(y).float(), torch.from_numpy(z).float()

    w_mx = torch.linalg.vector_norm(z.repeat(n,1,1) - torch.swapaxes(z.repeat(n,1,1), 0, 1), ord = 1, dim = 2)
    sigma_w_train = torch.median(w_mx).item()

    u_mx = torch.linalg.vector_norm(y.repeat(n,1,1) - torch.swapaxes(y.repeat(n,1,1), 0, 1), ord = 1, dim = 2)
    sigma_u_train = torch.median(u_mx).item()

    v_mx = torch.linalg.vector_norm(x.repeat(n,1,1) - torch.swapaxes(x.repeat(n,1,1), 0, 1), ord = 1, dim = 2)
    sigma_v_train = 1.0 # torch.median(v_mx).item()

    test_size = math.floor(n/k)
    stat_all = torch.zeros(k, 1)
    boot_temp_all = torch.zeros(k, boot_num)
    cur_k = 0


    for k_fold in range(k):
        torch.manual_seed(set_seed)
        k_fold_start = int(n/k * k_fold)
        k_fold_end = int(n/k * (k_fold+1))

        X_test, Y_test, Z_test = x[k_fold_start:k_fold_end], y[k_fold_start:k_fold_end], z[k_fold_start:k_fold_end]
        X_train, Y_train, Z_train = torch.cat((x[0:k_fold_start], x[k_fold_end:])), torch.cat((y[0:k_fold_start], y[k_fold_end:])), torch.cat((z[0:k_fold_start], z[k_fold_end:]))


        train_yz = DatasetSelect_GAN_ver2(Y_train, Z_train, batch_size)
        DataLoader_yz = torch.utils.data.DataLoader(train_yz, batch_size=batch_size, shuffle=True)

        train_xz = DatasetSelect_GAN_ver2(X_train, Z_train, batch_size)
        DataLoader_xz = torch.utils.data.DataLoader(train_xz, batch_size=batch_size, shuffle=True)

        if verbose:
            print("Training Y")

        G_zy = train_label(Z = Z_train, Y = Y_train, rounded = False,
                  noise_dimension = noise_dimension, noise_type = "normal", G_lr = G_zy_lr, hidden_layer_size = 512,
                  DataLoader = DataLoader_yz, BN_type = True, ReLU_coef = 0.5,
                  epochs_num=epcoh_y,  sigma_z = sigma_w_train, sigma_y = sigma_u_train,
                  lambda_1 = 1, wgt_decay = 1e-5,
                  lambda_3 = 1e-5, drop_out_p = 0.2, M_train = 20, verbose = verbose)

        if verbose:
            print("Training X")

        G_zx = train_label(Z = Z_train, Y = X_train, rounded = False,
                  noise_dimension = noise_dimension, noise_type = "normal", G_lr = G_zx_lr, hidden_layer_size = 512,
                  DataLoader = DataLoader_xz, BN_type = True, ReLU_coef = 0.5,
                  epochs_num=epcoh_x,  sigma_z = sigma_w_train, sigma_y = sigma_v_train,
                  lambda_1 = 1, wgt_decay = 1e-5,
                  lambda_3 = 1e-5, drop_out_p = 0.2, M_train = 20, verbose = verbose)

        G_zx = G_zx.eval()
        G_zy = G_zy.eval()

        Z_test, X_test, Y_test = Z_test[0:(test_size),:], X_test[0:(test_size),:], Y_test[0:(test_size),:]
        Z_test_repeat = Z_test.repeat(M,1).to(device)


        # Generate fake data
        Noise_fake = sample_noise(Z_test_repeat.shape[0], noise_dimension, "normal", input_var = 1.0/3.0).to(device)
        gen_y_all = G_zy(torch.cat((Z_test_repeat,Noise_fake),dim=1)).to(device)

        Noise_fake = sample_noise(Z_test_repeat.shape[0], noise_dimension, "normal", input_var = 1.0/3.0).to(device)
        gen_x_all = G_zx(torch.cat((Z_test_repeat,Noise_fake),dim=1)).to(device)

        gen_x_all = gen_x_all.reshape(test_size, M).detach().to(device)
        gen_y_all = gen_y_all.reshape(test_size, M).detach().to(device)

        gen_x_all = (gen_x_all> 0.9).float()

        z_all = Z_test.to(device)
        x_all = X_test.to(device)
        y_all = Y_test.to(device)


        w_mx = torch.linalg.vector_norm(z_all.repeat(test_size,1,1) - torch.swapaxes(z_all.repeat(test_size,1,1), 0, 1), ord = 1, dim = 2)
        sigma_w = torch.median(w_mx).item()

        u_mx = torch.abs(y_all.repeat(1, test_size) - y_all.repeat(1, test_size).T)
        sigma_u = torch.median(u_mx).item()

        v_mx = torch.abs(x_all.repeat(1, test_size) - x_all.repeat(1, test_size).T)
        sigma_v = 1.0 # torch.median(v_mx).item()

        cur_stat, cur_boot_temp = get_p_value_stat_1(boot_num, M, test_size, gen_x_all.to(device), gen_y_all.to(device),
                                x_all.to(device), y_all.to(device), z_all.to(device), sigma_w, sigma_u, sigma_v,
                                boor_rv_type)
        stat_all[cur_k,:] = cur_stat
        boot_temp_all[cur_k,:] = torch.from_numpy(cur_boot_temp)
        cur_k = cur_k + 1

    p_val = np.mean(torch.mean(boot_temp_all, dim = 0).numpy() > torch.mean(stat_all).item() )
    return p_val


In [ ]:
# ccle_index_input = 9
ccle_threshold_input = 0.05
batch_size = 256
noise_dimension = 1
G_zy_lr = 2e-5
G_zx_lr = 2e-3
k = 2
M = 100
boot_num = 1000
boor_rv_type = "gaussian"

for ccle_index_input in range(10):

    p_val = mgan_ccle(ccle_index_input = ccle_index_input, ccle_threshold_input = ccle_threshold_input, batch_size = batch_size,
                noise_dimension = noise_dimension, G_zy_lr = G_zy_lr, G_zx_lr = G_zx_lr, k = k, M = M, boot_num = boot_num,
                boor_rv_type = boor_rv_type, verbose = True, epcoh_y = 5000, epcoh_x = 5000, set_seed = 42)

    print(f'ccle_index [{ccle_index_input}], p_value [{p_val}]')

Genetic features dimension = 1638 on 1030 cancer cells
PLX4720: Cell number = 474
466 (1638,) [0.00793665 0.03243895 0.0438247  ... 0.04631409 0.51928232 0.07857439]
Training Y
Epoch [0],MSE [0.0348055399954319] MMD_loss [54.37702560424805]


 20%|██        | 1005/5000 [00:29<03:16, 20.38it/s]

Epoch [1000/5000], MSE [0.057580478489398956] MMD_loss [-0.11196421831846237]


 40%|████      | 2006/5000 [01:00<02:39, 18.74it/s]

Epoch [2000/5000], MSE [0.05623731017112732] MMD_loss [-0.11982018500566483]


...

 80%|████████  | 4004/5000 [01:55<00:59, 16.65it/s]

Epoch [4000/5000], MSE [0.10126549750566483] MMD_loss [0.03290821611881256]


100%|██████████| 5000/5000 [02:23<00:00, 34.84it/s]

Epoch [5000/5000], MSE [0.10126557946205139] MMD_loss [0.03287840634584427]


ccle_index [9], p_value [0.013]
